# Test model
Handling Panns cnn14

In [1]:
from audiointerp.dataset.esc50 import ESC50dataset
from audiointerp.model.cnn14 import TransferCnn14
from audiointerp.fit import Trainer
from audiointerp.processing.spectrogram import LogMelSTFTSpectrogram
import torch.nn as nn
import torch.optim as optim
import torchaudio.transforms as T
import torch
from torch.utils.data import DataLoader

In [2]:
root_dir = "/home/yuliya/ESC50"
sr = 32000
train_folds = [1, 2, 3]
valid_folds = [4]
test_folds = [5]

In [3]:
n_fft = 1024
hop_length = 320
win_length = 1024
n_mels = 64
f_min = 50
f_max = 14000
top_db = 80

In [4]:
feature_extractor = LogMelSTFTSpectrogram(
    n_fft=n_fft, win_length=win_length, hop_length=hop_length,
    sample_rate=sr, n_mels=n_mels, f_min=f_min, f_max=f_max, top_db=top_db,
    return_phase=False, return_full_db=False
)

In [5]:
train_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=train_folds, feature_extractor=feature_extractor)
valid_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=valid_folds, feature_extractor=feature_extractor)
test_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=test_folds, feature_extractor=feature_extractor)

In [6]:
train_loader_kwargs = {"batch_size": 32, "shuffle": True}
valid_loader_kwargs = {"batch_size": 32, "shuffle": False}
test_loader_kwargs = {"batch_size": 32, "shuffle": False}

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cls = TransferCnn14
model_kwargs = {"num_classes": 50}
model_pretrain_weights_path = "weights/Cnn14_mAP=0.431.pth"

optimizer_cls = optim.Adam
optimizer_kwargs = {"lr": 1e-3}

criterion_cls = nn.CrossEntropyLoss

In [8]:
model_trainer = Trainer(
    model_cls=model_cls,
    train_data=train_data,
    train_loader_kwargs=train_loader_kwargs,
    criterion_cls=criterion_cls,
    optimizer_cls=optimizer_cls,
    model_kwargs=model_kwargs,
    model_pretrain_weights_path=model_pretrain_weights_path,
    optimizer_kwargs=optimizer_kwargs,
    device=device,
    valid_data=valid_data,
    valid_loader_kwargs=valid_loader_kwargs,
    test_data=test_data,
    test_loader_kwargs=test_loader_kwargs
)

Random seed set to: 42


In [9]:
train_losses, train_accs, val_losses, val_accs, test_loss, test_acc = model_trainer.train(num_epochs=2, test=True)

Epoch 1/2:  50%|█████     | 1/2 [00:43<00:43, 43.59s/it]

Train Loss: 2.3053, Train Acc: 0.4100
Valid Loss: 0.6558, Valid Acc: 0.8125


Epoch 1/2: 100%|██████████| 2/2 [01:30<00:00, 45.16s/it]


Train Loss: 0.5176, Train Acc: 0.8350
Valid Loss: 0.3234, Valid Acc: 0.9075
Best val Acc: 0.9075
Модель сохранена в best.pth
Test Loss: 0.4259, Test Acc: 0.8725


In [10]:
model_trainer2 = Trainer(
    model_cls=model_cls,
    train_data=train_data,
    train_loader_kwargs=train_loader_kwargs,
    criterion_cls=criterion_cls,
    optimizer_cls=optimizer_cls,
    model_kwargs=model_kwargs,
    model_pretrain_weights_path=model_pretrain_weights_path,
    optimizer_kwargs=optimizer_kwargs,
    device=device,
    valid_data=valid_data,
    valid_loader_kwargs=valid_loader_kwargs,
    test_data=test_data,
    test_loader_kwargs=test_loader_kwargs
)

Random seed set to: 42


In [11]:
train_losses, train_accs, val_losses, val_accs, test_loss, test_acc = model_trainer2.train(num_epochs=2, test=True)

Epoch 1/2:  50%|█████     | 1/2 [00:45<00:45, 45.37s/it]

Train Loss: 2.3053, Train Acc: 0.4100
Valid Loss: 0.6558, Valid Acc: 0.8125


Epoch 1/2: 100%|██████████| 2/2 [01:35<00:00, 47.85s/it]


Train Loss: 0.5176, Train Acc: 0.8350
Valid Loss: 0.3234, Valid Acc: 0.9075
Best val Acc: 0.9075
Модель сохранена в best.pth
Test Loss: 0.4259, Test Acc: 0.8725
